In [1]:
import tensorflow as tf
import os
import datetime
import tensorflow_datasets as tfds
import tqdm
%load_ext tensorboard

In [2]:
print(tf.__version__)

2.7.0


In [3]:
dataset, info = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', with_info=True)
train_dataset = dataset['train']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0.incompleteJLZV3A/amazon_us_reviews-train.tfrecord


  0%|          | 0/104975 [00:00<?, ? examples/s]

Dataset amazon_us_reviews downloaded and prepared to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0. Subsequent calls will reuse this data.


In [4]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [5]:
len(list(train_dataset))

104975

In [6]:
BUFFER_SIZE = 30000
BATCH_SIZE = 128

In [7]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration = False)

In [8]:
for reviews in train_dataset.take(2):
  print(reviews)

{'data': {'customer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'19430629'>, 'helpful_votes': <tf.Tensor: shape=(), dtype=int32, numpy=0>, 'marketplace': <tf.Tensor: shape=(), dtype=string, numpy=b'US'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'Mobile_Electronics'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'B0044BZLCS'>, 'product_parent': <tf.Tensor: shape=(), dtype=string, numpy=b'286427579'>, 'product_title': <tf.Tensor: shape=(), dtype=string, numpy=b'Durable Smoke Shining Silicone Skin for Apple iPod Touch 4th Generation with Clear Screen Protector for itouch 4th gen + USB Wall Charger for iPod Touch 4G and Wristband'>, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b"PROS: Fast shipping, easy to install lcd protector  CONS: Poor quality case (you get what you pay for)    The product arrived (in GUAM) in less than a week, that is fast! The AC adapter looks and feels cheap. The screen protector and the silicon case are what I was 

In [9]:
for reviews in train_dataset.take(10):
  review_text = reviews['data']
  print(review_text.get('review_body').numpy())
  print(review_text.get('star_rating'))
  print(tf.where(review_text.get('star_rating')>3,1,0).numpy())

b"PROS: Fast shipping, easy to install lcd protector  CONS: Poor quality case (you get what you pay for)    The product arrived (in GUAM) in less than a week, that is fast! The AC adapter looks and feels cheap. The screen protector and the silicon case are what I was interested in. The screen was easy to install. Just make sure that the lcd is clean and free from any dust or lint, putting the protector on is perfectly easy. I had difficulty putting the case on. I placed the bottom part first but the top is kinda loose, so I tried putting the top first; the problem now is the bottom. So I placed the ipod on top of the case and let the silicon hold the back. Then I pulled the corners - it was a perfect fit except for the bottom front which I expected not to be tight, was a little loose. Over all it's a good protection for your ipod touch at a not so expensive price...    If you're looking for a good protection and don't have the budget, this is a decent choice. But if you have a couple o

In [10]:
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()

for _,reviews in tqdm.tqdm(train_dataset.enumerate()):
  review_text = reviews['data']
  reviews_tokens = tokenizer.tokenize(review_text.get('review_body').numpy())
  vocabulary_set.update(reviews_tokens)

vocab_size = len(vocabulary_set)
vocab_size

100%|██████████| 104975/104975 [01:21<00:00, 1281.18it/s]


73738

In [11]:
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set)

In [12]:
def encode(text_tensor, label_tensor):
  encoded_text = encoder.encode(text_tensor.numpy())
  label = tf.where(label_tensor>3,1,0)
  return encoded_text, label
  

In [15]:
def encode_map_fn(tensor):
  text = tensor['data'].get('review_body')
  label = tensor['data'].get('star_rating')

  encoded_text, label = tf.py_function(encode, inp = [text, label], Tout = (tf.int64, tf.int32))
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label

In [16]:
ar_encoded_data = train_dataset.map(encode_map_fn)

In [17]:
for i,j in ar_encoded_data.take(2):
  print(i)
  print(j)

tf.Tensor(
[69745 63468   457 69364 32244 72248 55779 45928  7532 71337 65158 12545
 48146 28107 62168 48146 71310 70140 43950 30398 10682 59335 26426 59335
 66321 72726  6485 37330 41206 34763 34492 43950 55078 69302  1190 18786
 43193  1389 43950 15199 45928 18786 14694 61553 12545 41573 62168 30716
 68829 18013 59335 43950 15199 68829 69364 32244 72248 69165 71401 28348
 41206 14694 55779 34763 24301 18786 17427 44033 16827 15339 62762 44354
 57430 14694 45928  1277 34763 15526 69364 30716 67366 56896 57430 14694
 12545  1277 30716  3353 14694 10233 63146 59287 41153 14694 53855 34763
 40754  3012 71701 30716 28479 57430 14694 53855 59287 14694 30860  9227
 34763 14694 10233  1347 30716  3353 14694 43998  1277 53855  8543 14694
 12545 18786 58930 14694 61553 69250 14694 46002 11769 30716 66300 14694
 70809 23494 68829  6485 19044 44174 70095 70140 14694 10233 70355   844
 30716 21427 28795 32244 10486 64489 68829  6485 25157  3012 70406 29887
 23494 56801  6485 22433  4021 70140 227

In [18]:
TAKE_SIZE = 1000

train_data = ar_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = ar_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

In [19]:
vocab_size += 1

In [20]:
sample_text, sample_labels = next(iter(test_data))
sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(2421,), dtype=int64, numpy=array([69745, 63468,   457, ...,     0,     0,     0])>,
 <tf.Tensor: shape=(), dtype=int32, numpy=0>)

In [21]:
for fo,f1 in test_data.take(10):
  print(tf.unique_with_counts(f1)[2].numpy()) #num of pos & neg reviews in the batch

[43 85]
[95 33]
[87 41]
[50 78]
[42 86]
[39 89]
[87 41]
[42 62]


### Building the model

In [23]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 128))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(1)) #sigmoid not used to get a linear activation and we can add a cutoff later, this would be faster

In [25]:
logdir = os.path.join("/tmp/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [26]:
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='tmp/sentiment_analysis.hdf5', verbose = 1, save_best_only=True)

In [27]:
model.compile(optimizer='adam', loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy']) #bec we are using linear layer -> logits

In [ ]:
history = model.fit(train_data, epochs =4, validation_data=test_data, callbacks=[tensorboard_callback, checkpointer])

Epoch 1/4
 78/813 [=>............................] - ETA: 8:18 - loss: 0.5257 - accuracy: 0.7100

In [ ]:
model.save('/tmp/final_sentiment_analysis')